# MLOps: Initial - Manual deployment & validation 

<div class="alert alert-warning"> 
	⚠️ <strong> PRE-REQUISITE: </strong> Before proceeding with this notebook, please ensure that you have executed the <code>1-data-prep-feature-store.ipynb</code> and <code>2-training-registry.ipynb</code> Notebooks</li>
</div>

## Contents

- [Introduction](#Introduction)
- [SageMaker Endpoint](#SageMaker-Endpoint)

## Introduction

This is our third notebook which will explore the model deployment of ML workflow.

Here, we will put on the hat of a `Data Scientist` or `ML Engineer` and will perform the task of model deployment which includes fetching the right model and deploying it for inference. This is often done during model development to validate consuming applications or inference code as well as for ad-hoc inference.  In the early stages of MLOps adoptions, this is often a manual task.  However, for models deployed that are serving live business critical production traffic - typically the deployment is handled through Infrastructure-as-Code(IaC)/Configuration-as-Code(CaC) outside the experimentation environment.  This allows for robust deployment patterns that support advanced deployment strategies (ex. A/B, Shadow, Blue/Green) as well as automatic rollback capabilities.

For this task we will be using Amazon SageMaker Model Hosting capabilities to manually deploy and perform inference.  In this  case, we will utilize a real-time endpoint; however, SageMaker has other deployment options to meet the needs of your use cases. Please refer to the provided [detailed guidance](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html) when choosing the right option for your use case. We'll also pull features for inference from the online SageMaker Feature Store.   Keep in mind, this is typically done through the consuming application and inference pipelines but we are showing it here to illustrate the manual steps often performed during validation stages of development. 

![Notebook3](images/Notebook-3.png)

Let's get started!

**Imports**

In [2]:
!pip install -U sagemaker

  Using cached sagemaker-2.169.0-py2.py3-none-any.whl
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.155.0
    Uninstalling sagemaker-2.155.0:
      Successfully uninstalled sagemaker-2.155.0
ERROR: pip's dependency resolver does not currently take into account 

In [3]:
%store -r

In [4]:
from urllib.parse import urlparse
import io
import time
from sagemaker.model import ModelPackage
import boto3
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import numpy as np
import pathlib
from sagemaker.feature_store.feature_group import FeatureGroup
from helper_library import *

**Session variables**

In [5]:
# Useful SageMaker variables
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role_arn= sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
s3_client = boto3.client('s3', region_name=region)
sagemaker_client = boto3.client('sagemaker')

## SageMaker Endpoint

You can deploy your trained model as [SageMaker hosted endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) which serves real-time predictions from a trained model. The endpoint will retrieve the model created during training and deploy it within a SageMaker scikit-learn container. This all can be accomplished with one line of code. Note that it will take several minutes to deploy the model to a hosted endpoint.

Let's get the model we registered in the Model Registry.

In [17]:
xgb_regressor_model = ModelPackage(
    role_arn,
    model_package_arn=model_package_arn,
    name=model_name
)

It's current status is `PendingApproval`. In order to use this model for offline predictions or as a real-time endpoint, we should adopt the practice of setting the status to `Approved` prior to deployment.   You can optionally register your model in `Approved` status at the time of model registration; however, this option allows for as secondary peer review approval workflow.  

In [18]:
sagemaker_client.update_model_package(
    ModelPackageArn=xgb_regressor_model.model_package_arn,
    ModelApprovalStatus='Approved'
)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:523914011708:model-package/synthetic-housing-models-ray-1688937647-c23c/2',
 'ResponseMetadata': {'RequestId': '05ffa9c8-d2ac-439a-b809-28da890fb9d9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '05ffa9c8-d2ac-439a-b809-28da890fb9d9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '123',
   'date': 'Tue, 11 Jul 2023 17:18:28 GMT'},
  'RetryAttempts': 0}}

Now we can deploy it!

In [19]:
from IPython.core.display import display, HTML
endpoint_name = f'{model_name}-endpoint-' + time.strftime('%Y-%m-%d-%H-%M-%S')
display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">The Endpoint</a> After About 5 Minutes</b>'.format(
            region, endpoint_name
        )
    )
)
xgb_regressor_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name
)

/tmp/ipykernel_2058/1749335991.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Using already existing model: xgboost-model


--------!

Let's test this real-time endpoint by passing it some data and getting a real-time prediction back.

## Read from offline Feature Store

Pull data from the offline feature store for quick validation of model deployment

In [20]:
train_feature_group_name = 'fs-train-synthetic-housing-data'
validation_feature_group_name = 'fs-val-synthetic-housing-data'
test_feature_group_name = 'fs-test-synthetic-housing-data'

In [21]:
fs_group = FeatureGroup(name=test_feature_group_name, sagemaker_session=sagemaker_session)  
query = fs_group.athena_query()
table = query.table_name
query_string = f'SELECT {features_to_select} FROM "sagemaker_featurestore"."{table}" ORDER BY record_id'
query_results = 'sagemaker-featurestore'
output_location = f's3://{bucket}/{query_results}/query_results/'
query.run(
    query_string=query_string, 
    output_location=output_location
)
query.wait()
df = query.as_dataframe()
df.head()

,price,year_built,square_feet,num_bedrooms,num_bathrooms,lot_acres,garage_spaces,front_porch,deck
0,271801,0.737282,-1.653984,-1.432323,-0.696323,0.976884,0.427544,-0.987742,0.984782
1,271801,0.737282,-1.653984,-1.432323,-0.696323,0.976884,0.427544,-0.987742,0.984782
2,391223,-0.851152,0.073506,-1.432323,0.727893,-0.689770,0.427544,-0.987742,-1.015453
3,391223,-0.851152,0.073506,-1.432323,0.727893,-0.689770,0.427544,-0.987742,-1.015453
4,427149,-0.950429,0.786574,-1.432323,-0.696323,1.214977,-1.348341,-0.987742,-1.015453


In [22]:
# Attach to the SageMaker endpoint
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer(),
    deserializer=JSONDeserializer()
)

dropped_df = df.drop(columns=["price"])

# Get a real-time prediction (only predicting the 1st 5 row to reduce output size)
predictor.predict(dropped_df[:5].to_csv(index=False, header=True))

[279469.73818449036,
 279469.73818449036,
 404231.97742961714,
 404231.97742961714,
 432126.5458102586]

## Read from online Feature Store

In a production environment supporting real-time use cases, the ability to pull features for inference with low latency is critical.   Here, we show how you can pull records from the online feature store to use with your deployed real-time endpoint. Again, this is typically done through the consuming application or inference pipeline but we show it here to illustrate the required steps. 

In [29]:
customer_record = get_online_feature_group_records(test_feature_group_name, ['1', '2', '3', '4'])

In [30]:
customer_record

[{'PRICE': 446178,
  'YEAR_BUILT': 0.8427086385842989,
  'SQUARE_FEET': -0.28251595276670355,
  'NUM_BEDROOMS': -0.015507900644514884,
  'NUM_BATHROOMS': 0.7101229270627583,
  'LOT_ACRES': 0.04875641577808017,
  'GARAGE_SPACES': -0.43577312821090286,
  'FRONT_PORCH': 1.0239534706909195,
  'DECK': 1.0010005005003753,
  'event_time': 1686172277.0,
  'record_id': 2},
 {'PRICE': 503543,
  'YEAR_BUILT': 0.2366008930139014,
  'SQUARE_FEET': 0.5454168652378967,
  'NUM_BEDROOMS': -0.015507900644514884,
  'NUM_BATHROOMS': 1.422263585992484,
  'LOT_ACRES': 0.8876623816442674,
  'GARAGE_SPACES': -1.3190970367465167,
  'FRONT_PORCH': 1.0239534706909195,
  'DECK': 1.0010005005003753,
  'event_time': 1686172277.0,
  'record_id': 3},
 {'PRICE': 310304,
  'YEAR_BUILT': -0.47052481015156233,
  'SQUARE_FEET': -1.1263312378273485,
  'NUM_BEDROOMS': 0.6893966741061539,
  'NUM_BATHROOMS': 0.7101229270627583,
  'LOT_ACRES': -0.5904100344056817,
  'GARAGE_SPACES': 0.4475507803247109,
  'FRONT_PORCH': -0.9766

In [39]:
record = customer_record[1]

In [40]:
record

{'PRICE': 503543,
 'YEAR_BUILT': 0.2366008930139014,
 'SQUARE_FEET': 0.5454168652378967,
 'NUM_BEDROOMS': -0.015507900644514884,
 'NUM_BATHROOMS': 1.422263585992484,
 'LOT_ACRES': 0.8876623816442674,
 'GARAGE_SPACES': -1.3190970367465167,
 'FRONT_PORCH': 1.0239534706909195,
 'DECK': 1.0010005005003753,
 'event_time': 1686172277.0,
 'record_id': 3}

In [41]:
record.pop('PRICE')
record.pop('event_time')
record.pop('record_id')

3

In [42]:
record

{'YEAR_BUILT': 0.2366008930139014,
 'SQUARE_FEET': 0.5454168652378967,
 'NUM_BEDROOMS': -0.015507900644514884,
 'NUM_BATHROOMS': 1.422263585992484,
 'LOT_ACRES': 0.8876623816442674,
 'GARAGE_SPACES': -1.3190970367465167,
 'FRONT_PORCH': 1.0239534706909195,
 'DECK': 1.0010005005003753}

In [43]:
from io import StringIO
import pandas as pd
pd.read_csv(StringIO('-1.2786684709120923,-0.2238534355557023,-1.4253170501458523,-0.0020177318669674,-0.6703058406786521,1.330874688860325,-0.9766068758234135,1.0010005005003753')).shape == (0,8)

True

In [44]:
payload = ','.join(str(record[key]) for key in customer_record[1])

In [45]:
payload

'0.2366008930139014,0.5454168652378967,-0.015507900644514884,1.422263585992484,0.8876623816442674,-1.3190970367465167,1.0239534706909195,1.0010005005003753'

Predict using the features pulled from the online feature store...

In [46]:
predictor.predict('-1.2786684709120923,-0.2238534355557023,-1.4253170501458523,-0.0020177318669674,-0.6703058406786521,1.330874688860325,-0.9766068758234135,1.0010005005003753')

[340175.03465732496]

In [47]:
predictor.predict(payload)

[500700.23876614915]